<a href="https://colab.research.google.com/github/german1728/Chimera/blob/master/Copia_de_Tutorial5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 5
# Selección de modelo: compensación de sesgo-varianza

**Content creators**: Pierre-Étienne Fiquet, Anqi Wu, Alex Hyafil with help from Ella Batty

**Content reviewers**: Lina Teichmann, Patrick Mineault, Michael Waskom

**Content traductor**: Israel Chaparro

**Source**: Neuromatch Academy



---
#Objetivos del tutorial

Este es el Tutorial 5 de una serie sobre cómo ajustar modelos a datos. Comenzamos con regresión lineal simple, usando optimización de mínimos cuadrados (Tutorial 1) y Estimación de máxima verosimilitud (Tutorial 2). Usaremos bootstrapping para construir intervalos de confianza alrededor de los parámetros del modelo lineal inferido (Tutorial 3). Terminaremos nuestra exploración de modelos de regresión generalizando a la regresión lineal múltiple y la regresión polinomial (Tutorial 4). Terminamos aprendiendo a elegir entre estos distintos modelos. Analizamos la compensación de sesgo-varianza (Tutorial 5) y la Validación cruzada para la selección del modelo (Tutorial 6).

En este tutorial, aprenderemos sobre la compensación de sesgo-varianza y la veremos en acción usando modelos de regresión polinomial.

Objetivos del tutorial:

* Comprender la diferencia entre la prueba y los datos del tren.
* Compare el tren y el error de prueba para modelos de diversa complejidad
* Comprender cómo se relaciona el equilibrio entre sesgo y varianza con el modelo que elegimos

---
# Configuracion

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#@title Configuracion de Figuras
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

In [ ]:
#@title Funciones de Ayuda
def ordinary_least_squares(x, y):
  """Ordinary least squares estimator for linear regression.

  Args:
    x (ndarray): design matrix of shape (n_samples, n_regressors)
    y (ndarray): vector of measurements of shape (n_samples)

  Returns:
    ndarray: estimated parameter values of shape (n_regressors)
  """

  return np.linalg.inv(x.T @ x) @ x.T @ y

def make_design_matrix(x, order):
  """Create the design matrix of inputs for use in polynomial regression

  Args:
    x (ndarray): input vector of shape (n_samples)
    order (scalar): polynomial regression order

  Returns:
    ndarray: design matrix for polynomial regression of shape (samples, order+1)
  """

  # Broadcast to shape (n x 1) so dimensions work
  if x.ndim == 1:
    x = x[:, None]

  #if x has more than one feature, we don't want multiple columns of ones so we assign
  # x^0 here
  design_matrix = np.ones((x.shape[0],1))

  # Loop through rest of degrees and stack columns
  for degree in range(1, order+1):
      design_matrix = np.hstack((design_matrix, x**degree))

  return design_matrix


def solve_poly_reg(x, y, max_order):
  """Fit a polynomial regression model for each order 0 through max_order.

  Args:
    x (ndarray): input vector of shape (n_samples)
    y (ndarray): vector of measurements of shape (n_samples)
    max_order (scalar): max order for polynomial fits

  Returns:
    dict: fitted weights for each polynomial model (dict key is order)
  """

  # Create a dictionary with polynomial order as keys, and np array of theta
  # (weights) as the values
  theta_hats = {}

  # Loop over polynomial orders from 0 through max_order
  for order in range(max_order+1):

    X = make_design_matrix(x, order)
    this_theta = ordinary_least_squares(X, y)

    theta_hats[order] = this_theta

  return theta_hats

---
# Sección 1: Datos de prueba vs tren

Los datos utilizados para el procedimiento de ajuste para un modelo dado son los **datos de entrenamiento**. En el tutorial 4, calculamos MSE sobre los datos de entrenamiento de nuestros modelos de regresión polinomial y comparamos el MSE de entrenamiento entre modelos. Un tipo de datos adicional importante son **datos de prueba**. Estos son datos retenidos que no se utilizan (de ninguna manera) durante el procedimiento de ajuste. Al ajustar modelos, a menudo queremos considerar tanto el error del tren (la calidad de la predicción en los datos de entrenamiento) como el error de la prueba (la calidad de la predicción en los datos de la prueba), como veremos en la siguiente sección.

Generaremos algunos datos ruidosos para usar en este tutorial usando un proceso similar al del Tutorial 4, sin embargo, ahora también generaremos datos de prueba. Queremos ver cómo nuestro modelo se generaliza más allá del rango de valores que vemos en la fase de entrenamiento. Para lograr esto, generaremos x a partir de un rango más amplio de valores ([-3, 3]). Luego trazamos el tren y los datos de prueba juntos.

In [ ]:
#@title
#@markdown Ejecute esta celda para simular tanto los datos de entrenamiento como los de prueba

### Generate training data
np.random.seed(0)
n_train_samples = 50
x_train = np.random.uniform(-2, 2.5, n_train_samples) # sample from a uniform distribution over [-2, 2.5)
noise = np.random.randn(n_train_samples) # sample from a standard normal distribution
y_train =  x_train**2 - x_train - 2 + noise

### Generate testing data
n_test_samples = 20
x_test = np.random.uniform(-3, 3, n_test_samples) # sample from a uniform distribution over [-2, 2.5)
noise = np.random.randn(n_test_samples) # sample from a standard normal distribution
y_test =  x_test**2 - x_test - 2 + noise

## Plot both train and test data
fig, ax = plt.subplots()
plt.title('Training & Test Data')
plt.plot(x_train, y_train, '.', markersize=15, label='Training')
plt.plot(x_test, y_test, 'g+', markersize=15, label='Test')
plt.legend()
plt.xlabel('x')
plt.ylabel('y');

---
# Sección 2: Compensación de sesgo-varianza

Encontrar un buen modelo puede resultar complicado. Uno de los conceptos más importantes a tener en cuenta al modelar es la **compensación entre sesgo y varianza**.

**Sesgo** es la diferencia entre la predicción del modelo y las correspondientes variables de salida verdaderas que está intentando predecir. Los modelos con alto sesgo no se ajustarán bien a los datos de entrenamiento ya que las predicciones son bastante diferentes de los datos reales. Estos modelos de alto sesgo están demasiado simplificados: no tienen suficientes parámetros y complejidad para capturar con precisión los patrones en los datos y, por lo tanto, **no se ajustan correctamente**.


**Varianza** se refiere a la variabilidad de las predicciones del modelo para una entrada determinada. Esencialmente, ¿las predicciones del modelo cambian mucho con los cambios en los datos de entrenamiento exactos utilizados? Los modelos con una alta varianza dependen en gran medida de los datos de entrenamiento exactos utilizados; no se generalizarán bien para probar los datos. Estos modelos de alta varianza están **sobreajustados** a los datos.

En esencia:

* Los modelos de alto sesgo y baja varianza tienen un alto error de tren y de prueba.
* Bajo sesgo, los modelos de alta varianza tienen un error de tren bajo, error de prueba alto
* Los modelos de bajo sesgo y baja varianza tienen un tren y un error de prueba bajos

Como podemos ver en esta lista, ¡idealmente queremos modelos de bajo sesgo y baja varianza! Sin embargo, estos objetivos pueden entrar en conflicto: los modelos con la complejidad suficiente para tener un sesgo bajo también tienden a sobreajustarse y dependen más de los datos de entrenamiento. Necesitamos decidir cuál es la compensación correcta.

En esta sección, veremos la compensación de sesgo-varianza en acción con modelos de regresión polinomial de diferentes órdenes.

Ilustración gráfica de sesgo y varianza.
(Fuente: http://scott.fortmann-roe.com/docs/BiasVariance.html)

![sesgo-varianza](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/images/bias_variance/bullseye.png)

Primero ajustaremos modelos de regresión polinomial de órdenes 0-5 en nuestros datos de entrenamiento simulados tal como hicimos en el Tutorial 4.

In [ ]:
#@title
#@markdown Ejecute esta celda para estimar theta_hats
max_order = 5
theta_hats = solve_poly_reg(x_train, y_train, max_order)

### Ejercicio 1: Calcule y compare el error del tren con el de la prueba

Usaremos MSE como nuestra métrica de error nuevamente. Calcule MSE en datos de entrenamiento ($ x_{train}, y_{train} $) y datos de prueba ($ x_{test}, y_{test} $) para cada modelo de regresión polinomial (órdenes 0-5). Dado que ya desarrolló código en el Ejercicio 4 de T4 para evaluar polinomios de ajuste, lo hemos trasladado aquí a la función ``eval_poly_reg`` para su uso.

In [ ]:
def evaluate_poly_reg(x, y, theta_hats, max_order):
    """ Evaluates MSE of polynomial regression models on data

    Args:
      x (ndarray): input vector of shape (n_samples)
      y (ndarray): vector of measurements of shape (n_samples)
      theta_hats (dict):  fitted weights for each polynomial model (dict key is order)
      max_order (scalar): max order of polynomial fit

    Returns
      (ndarray): mean squared error for each order, shape (max_order)
    """

    mse = np.zeros((max_order + 1))
    for order in range(0, max_order + 1):
      X_design = make_design_matrix(x, order)
      y_hat = np.dot(X_design, theta_hats[order])
      residuals = y - y_hat
      mse[order] = np.mean(residuals ** 2)

    return mse

In [ ]:
def compute_mse(x_train,x_test,y_train,y_test,theta_hats,max_order):
  """Compute MSE on training data and test data.

  Args:
    x_train(ndarray): training data input vector of shape (n_samples)
    x_test(ndarray): test data input vector of shape (n_samples)
    y_train(ndarray): training vector of measurements of shape (n_samples)
    y_test(ndarray): test vector of measurements of shape (n_samples)
    theta_hats(dict): fitted weights for each polynomial model (dict key is order)
    max_order (scalar): max order of polynomial fit

  Returns:
    ndarray, ndarray: MSE error on training data and test data for each order
  """

  mse_train = ...
  mse_test = ...

  return mse_train, mse_test


#Descomente a continuación para probar su función
# mse_train, mse_test = compute_mse(x_train, x_test, y_train, y_test, theta_hats, max_order)

fig, ax = plt.subplots()
width = .35

# ax.bar(np.arange(max_order + 1) - width / 2, mse_train, width, label="train MSE")
# ax.bar(np.arange(max_order + 1) + width / 2, mse_test , width, label="test MSE")

ax.legend()
ax.set(xlabel='Polynomial order', ylabel='MSE', title ='Comparing polynomial fits');

[*Haga clic para obtener una solución*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D3_ModelFitting/solutions/W1D3_Tutorial5_Solution_b3fdca6e.py)

*Salida de ejemplo:*

<img alt='Solution hint' align='left' width=558 height=414 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W1D3_ModelFitting/static/W1D3_Tutorial5_Solution_b3fdca6e_0.png>



Como podemos ver en el gráfico anterior, los modelos más complejos (polinomios de orden superior) tienen un MSE más bajo para los datos de entrenamiento. Los modelos demasiado simplificados (órdenes 0 y 1) tienen un alto MSE en los datos de entrenamiento. A medida que agregamos complejidad al modelo, pasamos de un sesgo alto a un sesgo bajo.

El MSE de los datos de prueba sigue un patrón diferente. El mejor MSE de prueba es para un modelo de orden 2; esto tiene sentido ya que los datos se generaron con un modelo de orden 2. Tanto los modelos más simples como los modelos más complejos tienen un MSE de prueba más alto.

Entonces, para recapitular:

Modelo de orden 0: alto sesgo, baja varianza

Modelo de orden 5: sesgo bajo, alta varianza

Modelo de orden 2: justo, bajo sesgo, baja varianza

---
# Resumen

- Los datos de entrenamiento son los datos utilizados para el ajuste, los datos de prueba son datos retenidos.
- Necesitamos encontrar el equilibrio adecuado entre sesgo y varianza. Idealmente, queremos encontrar un modelo con una complejidad de modelo óptima que tenga tanto sesgo bajo como varianza baja
  - Los modelos demasiado complejos tienen un sesgo bajo y una gran varianza.
  - Los modelos demasiado simples tienen un sesgo alto y una varianza baja.